# Assignment 01: Basket analysis

Today there are several open source libraries to implement a basket analysis. One of the most known library is the Apriori algorithm that we practiced in PS03.

In this assignment, we will develop our own association rules algorithm and we will compare with the Apriori for a specific dataset.

To simplify things, our association rules algorithm will calculate relations between **pair of products** only, that is itemsets of size 2.

# 0. Preliminaries

## 0.1. Dataset

In this practice we are a new dataset contained in zip file `instacart_sample.zip`. **Please unzip this file first.** Within it, you will find a file with 10000 purchase transactions: `order_products.csv`. Each transaction is mainly represented by the *Order_Id* and the *Product_Id* of the purchased product. The original dataset has two other variables (*add_cart_order* and *reordered*) that you can simply ignore.

We will use a second file: `products.csv` that contains more details about each product, in particular the correspondence between a *Product_Id* and its name.

## 0.2. Imports

In [1]:
import pandas as pd
import numpy as np
import sys
from itertools import combinations, groupby
from collections import Counter
from IPython.display import display
from apyori import apriori

## 0.3. Load the data

Open the csv with separator "," and assign to a dataframe variable (use read_csv from Pandas library). Let's estimate the 

In [2]:
purchase_transactions = pd.read_csv('order_products.csv')
purchase_transactions.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


Let's calculate the type of the variables (integer, float, chart...) and the size of the dataset and the file.

In [3]:
purchase_transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32434489 entries, 0 to 32434488
Data columns (total 4 columns):
order_id             int64
product_id           int64
add_to_cart_order    int64
reordered            int64
dtypes: int64(4)
memory usage: 989.8 MB


As the number of transactions is more than 32 milions, we will select a sample of 1 milion to facilitate computations.

In [4]:
purchase_transactions_sample=purchase_transactions[0:1000000]

Let's check the current dimension of the order dataset.

In [5]:
purchase_transactions_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 4 columns):
order_id             1000000 non-null int64
product_id           1000000 non-null int64
add_to_cart_order    1000000 non-null int64
reordered            1000000 non-null int64
dtypes: int64(4)
memory usage: 30.5 MB


# 1. Data preparation

[**CODE**] Transform the `purchase_transaction_sample` dataframe to a Series named `purchase_transactions_sample_serie` where the **order_id** variable becomes the index and the variable **item_id** becomes the value. Others variables should be removed from the original `purchase_transaction_sample` dataframe.

[**REPORT**] Create a table with the first **20 positions** of the new purchase transactions serie. Explain the result.

At this point, your serie should look like this:

In [8]:
purchase_transactions_sample_serie.head(20)

order_id
2    33120
2    28985
2     9327
2    45918
2    30035
2    17794
2    40141
2     1819
2    43668
3    33754
3    24838
3    17704
3    21903
3    17668
3    46667
3    17461
3    32665
4    46842
4    26434
4    39758
Name: item_id, dtype: int64

[**CODE**] Verify that the type of the `purchase_transactions_sample_serie` is a **serie**.

Tip: use `type()` to verify the serie format.

[**REPORT**] How many unique purchase transactions we have? How many unique products we have?

# 2. Association rules algorithm

## 2.1. Support functions

In order to facilitate the implementation of the association rules algorithm, we provide three functions to be used later: 

Function `item_frequency`: returns the counts for each individual and pair items.

In [11]:
def item_frequency(data):
    if type(data) == pd.core.series.Series:
        return data.value_counts().rename("frequency")
    else: 
        return pd.Series(Counter(data)).rename("frequency")

Function `transactions_count`: returns the number of unique transactions.

In [12]:
def transactions_count(transaction_item):
    return len(set(transaction_item.index))

Function `get_item_pairs`: returns items pairs, one at a time

In [13]:
def get_item_pairs(order_item):
    order_item = order_item.reset_index().as_matrix()
    for order_id, order_object in groupby(order_item, lambda x: x[0]):
        item_list = [item[1] for item in order_object]
              
        for item_pair in combinations(item_list, 2):
            yield item_pair

Function `merge_items_statistics`: returns items pairs, one at a time.

In [14]:
def merge_item_statistics(item_pairs, item_statistics):
    item_pairs_merged_A=item_pairs.merge(item_statistics.rename(columns={'frequency': 'frequency_A', 'support': 'support_A'}), left_on='item_A', right_index=True)
    item_pairs_merged_A_B=item_pairs_merged_A.merge(item_statistics.rename(columns={'frequency': 'frequency_B', 'support': 'support_B'}), left_on='item_B', right_index=True)
    return item_pairs_merged_A_B

## 2.2. Association rules function

[**CODE**] Complete the `association_rules_function` that generates a table with the following fields:
- item A: first item of the association rule
- item B: second item of the association rule
- frequency_AB: counts of the itemset (item_A, item_B), i.e., number of time both items appear in a transaction
- support_AB: percentage of purchase transactions that contain the itemset (item_A, item_B)
- frequency_A:  number of times item_A appears in the purchase transactions
- support_A: percentage of purchase transactions that contain the item A
- frequency_B: number of times item_B appears in the purchase transactions
- support_B: percentage of purchase transactions that contain the item B
- confidence_A_B: percentage of times that item B is purchased, given that item A was purchased 
- confidence_B_A: percentage of times that item A is purchased, given that item B was purchased
- lift: lift{A,B} = lift{B,A} = support{A,B} / (support{A} * support{B})

In [15]:
def association_rules(transaction_item, min_support):
    
    # Calculate the frequency and support of each purchased product in the transaction input 
    # - create variable "item_statistics" with "frequency" and "support" attributes
    ## ADD YOUR CODE HERE
    
    # Print the first 10 purchased-items' statistics
    print(item_statistics[0:10])

    # Eliminate those items whose support< min_support
    # - create variables "correct_items" and "transaction_item_filtered"
    ## ADD YOUR CODE HERE

    print("The number of purchase items with support >=",min_support," is:",len(correct_items))
    print("The number of purchase transactions after filtering items with support below min_support is:", len(transaction_item_filtered))

    # Eliminate purchase transactions with less than 2 items
    # - create variables "correct_transaction" and "transaction_item_filtered"
    ## ADD YOUR CODE HERE
        
    print("The purchase transactions with at lease 2 items is formed by", len(correct_transaction), "transactions")
    print("The size of the purchase transaction dataset after filtering >=min_support and >=2 items is", len(transaction_item_filtered))

    # Recalculate item statistics, i.e. frequency and support
    # - create variable "new_item_statistics" with "frequency" and "support" attributes
    ## ADD YOUR CODE HERE

    # Now, call the get_item_pairs function to obtain the pairs items
    item_pair_gen= get_item_pairs(transaction_item_filtered)

    # For the new purchased item-pairs, calculate the statistics (i.e. frequency_AB and support_AB) and eliminate those not complaint in terms of min_support
    # - create variable "item_pairs" with attributes "frequency_AB" and "support_AB"
    # - create variable "item_pairs_filtered"
    ## ADD YOUR CODE HERE
    
    #Let's number of pair-items purchases
    print("The number of purchased AB item-pairs is:",len(item_pairs))
    print("The number of purchased ABitem-pairs with support_AB>=min_support is:", len(item_pairs_filtered))

    # Create a new dataframe (association_rules_items) with the final association rules
    # merged with the previous statistics and calculate confidence_A_B, confidence_B_A and lift
    # The "association_rules_items" dataframe will have the following attributes:
    # item_A, item_B, frequency_AB, support_AB, frequency_A, support_A, frequency_B, support_B, confidence_AB, confidence_BA, lift
    ## ADD YOUR CODE HERE
        
    # Order by descending the association_rules_items by lift value.
    # - create variable "association_rules_items_ordered"
    ## ADD YOUR CODE HERE
    
    return association_rules_items_ordered

[**REPORT**] Create a table with the top 5 association rules. Explain the results. Which are the purchased items with higher lift? What does it mean?.

If we calculate the association rules for min_support=0.01, your output should look like this:

In [16]:
min_support=0.01
association_rules_df = association_rules(purchase_transactions_sample_serie, min_support)  

       frequency    support
24852      14759  14.869031
13176      11867  11.955470
21137       8138   8.198670
21903       7477   7.532742
47209       6551   6.599839
47766       5379   5.419101
47626       4700   4.735039
16797       4388   4.420713
26209       4317   4.349184
27966       4238   4.269595
The number of purchase items with support >= 0.01  is: 11325
The number of purchase transactions after filtering items with support below min_support is: 927649
The purchase transactions with at lease 2 items is formed by 93142 transactions
The size of the purchase transaction dataset after filtering >=min_support and >=2 items is 922075


/Users/maca/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


The number of purchased AB item-pairs is: 3672671
The number of purchased ABitem-pairs with support_AB>=min_support is: 51856


In [17]:
association_rules_df.head()

,item_A,item_B,frequency_AB,support_AB,frequency_A,support_A,frequency_B,support_B,confidence_A_B,confidence_B_A,lift
39609,29479,16508,11,0.011810,29,0.031135,33,0.035430,0.379310,0.333333,10.705977
28389,519,49508,11,0.011810,33,0.035430,34,0.036503,0.333333,0.323529,9.131569
11051,23953,27553,12,0.012884,38,0.040798,33,0.035430,0.315789,0.363636,8.913110
45109,29126,36361,10,0.010736,40,0.042945,27,0.028988,0.250000,0.370370,8.624259
27365,48476,20631,10,0.010736,54,0.057976,21,0.022546,0.185185,0.476190,8.213580


Load the `products.csv` dataset that includes each purchased item.

In [18]:
purchased_item_name   = pd.read_csv('products.csv')

This `purchase_item_name` dataframe should look like this:

In [19]:
purchased_item_name.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


[**CODE**] Create a merge function (`merge_product_name_item_id(product_name, association_rules)` )that replaces item_A and item_B ids with corresponding product_name in the `association_rules`dataframe.

In [20]:

def merge_product_name_item_id(product_name, association_rules):
    columns = ['itemA','itemB','frequency_AB','support_AB','frequency_A','support_A','frequency_B','support_B', 
               'confidence_A_B','confidence_B_A','lift']
    #WRITE YOUR CODE
    
    return association_rules[columns]

Finally, we execute the `merge_product_name_item_id` to generate the association rules with product name.

In [21]:
association_rules_w_product_name = merge_product_name_item_id(purchased_item_name, association_rules_df).sort_values('lift', ascending=False)

[**REPORT**] Create a table with the top 5 product-pairs based on lift value. Explain the results according to the statistics results.

[**REPORT**] Create a table with the last 5 product-pairs based on lift value. Explain the results according to the statistics results.

At this point, your tables should look like this:

In [22]:
association_rules_w_product_name.head()

,itemA,itemB,frequency_AB,support_AB,frequency_A,support_A,frequency_B,support_B,confidence_A_B,confidence_B_A,lift
0,Gobble Gobble Turkey Pouch Stage 3,Stage 3 Hearty Meals - Chick Chick Organic Bab...,11,0.011810,29,0.031135,33,0.035430,0.379310,0.333333,10.705977
1,Antioxidant Infusions Brasilia Blueberry,Antioxidant Infusions Costa Rica Clementine,11,0.011810,33,0.035430,34,0.036503,0.333333,0.323529,9.131569
2,Cream on Top Strawberry Organic Yogurt,Organic Blueberry Cream On Top Whole Milk Yogurt,12,0.012884,38,0.040798,33,0.035430,0.315789,0.363636,8.913110
3,Organic Strawberry Chia Lowfat 2% Cottage Cheese,Organic Cottage Cheese Blueberry Acai Chia,10,0.010736,40,0.042945,27,0.028988,0.250000,0.370370,8.624259
4,Tender Chicken & Stars Stage 3,Organic Spring Vegetables & Pasta Stage 3 Baby...,10,0.010736,54,0.057976,21,0.022546,0.185185,0.476190,8.213580


In [23]:
association_rules_w_product_name.tail()

,itemA,itemB,frequency_AB,support_AB,frequency_A,support_A,frequency_B,support_B,confidence_A_B,confidence_B_A,lift
360,Strawberries,Organic Strawberries,23,0.024693,4352,4.672436,8096,8.692105,0.005285,0.002841,0.000608
3793,Organic Avocado,Organic Hass Avocado,12,0.012884,5368,5.763243,6537,7.018316,0.002235,0.001836,0.000319
8041,Organic Hass Avocado,Organic Avocado,11,0.011810,6537,7.018316,5368,5.763243,0.001683,0.002049,0.000292
1567,Banana,Bag of Organic Bananas,24,0.025767,14690,15.771618,11781,12.648429,0.001634,0.002037,0.000129
2637,Bag of Organic Bananas,Banana,16,0.017178,11781,12.648429,14690,15.771618,0.001358,0.001089,0.000086


[**REPORT**] Considering the previous results:

- As Data Scientist, which is your main recommendation to increase sales? Explain why
- When a customer purchases **Gobble Gobble Turkey Pouch Stage 3**, which is the product that uses to buy too? Why?
- Indicate two products that do **NOT** tend to appear together. Why? 

# Deliver

Deliver:

* A zip file containing your notebook (.ipynb file) with all the [**CODE**] parts implemented.
* A 4-pages PDF report including all parts of this notebook marked with "[**REPORT**]"

The report should end with the following statement: **We hereby declare that, except for the code provided by the course instructors, all of our code, report, and figures were produced by ourselves.**